In [25]:
import sys

sys.path.append("..")

import numpy as np
import datetime

from ex5.util import parse_dataset
from ex5.conformal import ConformalBoostedSeqTree
from ex5.conformal import custom_train_test_split


# DATASET_PATH = "../datasets/pioneer.txt"  ## 160 lines
#DATASET_PATH = "../datasets/auslan2.txt"  ## 200 lines
DATASET_PATH = "../datasets/context.txt"  ## 240 lines
# DATASET_PATH = "../datasets/aslbu.txt"  #### 424 lines
# DATASET_PATH = "../datasets/skating.txt"  ## 530 lines
# DATASET_PATH = "../datasets/reuters.txt"  # 1010 lines
# DATASET_PATH = "../datasets/webkb.txt"  ### 3667 lines
# DATASET_PATH = "../datasets/news.txt"  #### 4976 lines
# DATASET_PATH = "../datasets/unix.txt"  #### 5472 lines

# other datasets with 2 classes
# DATASET_PATH = "../datasets/activity.txt"  # 35 lines
# DATASET_PATH = "../datasets/question.txt"  # 1730 lines
# DATASET_PATH = "../datasets/epitope.txt"  # 2392 lines
# DATASET_PATH = "../datasets/gene.txt"  # 2942 lines
# DATASET_PATH = "../datasets/robot.txt"  # 4302 lines

ITERATIONS = 20

In [26]:
df = parse_dataset(DATASET_PATH, add_padding=True)

# divide training and test sets
X, X_test, Y, Y_test = custom_train_test_split(df["s"].to_numpy(), df["y"].to_numpy())

Class 1: 35 train, 9 test
Class 2: 38 train, 10 test
Class 3: 38 train, 10 test
Class 4: 40 train, 10 test
Class 5: 40 train, 10 test


In [27]:
print(len(X_test))
print(len(X))

49
191


In [28]:
model = ConformalBoostedSeqTree()
model.fit(X, Y)

class: 1
Training model for class 1
Fitting Classifier


In [ ]:



model.save(f"models/{DATASET_PATH.split('/')[-1].split('.')[0]}_itrs_{ITERATIONS}_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")


In [ ]:

#MODEL = "models_activity_itrs_20_2024-06-07_14-54-24.pkl"
#model.load(MODEL)


In [ ]:
SIGNIFICANCE = 0.2

In [ ]:

predictions = model.predict(X_test, SIGNIFICANCE)
for i in range(len(predictions)):
    print(f"Real: {Y_test[i]} - Predicted: {predictions[i]}")

Real: 1 - Predicted: ['1', '10', '2', '4', '5', '6', '7', '8']
Real: 1 - Predicted: ['1', '10', '2', '3', '4', '5', '6', '7', '8', '9']
Real: 1 - Predicted: ['1', '10', '2', '3', '4', '5', '6', '7', '8', '9']
Real: 1 - Predicted: ['1', '10', '2', '3', '4', '5', '6', '7', '8', '9']
Real: 10 - Predicted: ['1', '10', '2', '4', '5', '6', '7', '8', '9']
Real: 10 - Predicted: ['1', '2', '3', '4', '7', '8', '9']
Real: 10 - Predicted: ['1', '10', '2', '4', '6', '7', '8', '9']
Real: 10 - Predicted: ['1', '5', '7', '8', '9']
Real: 2 - Predicted: ['1', '10', '2', '3', '4', '5', '6', '7', '8', '9']
Real: 2 - Predicted: ['1', '10', '2', '4', '5', '7', '8', '9']
Real: 2 - Predicted: ['1', '2', '3', '4', '5', '6', '7', '8', '9']
Real: 2 - Predicted: ['1', '2', '3', '4', '6', '7', '8', '9']
Real: 3 - Predicted: ['10', '2', '3', '4', '5', '7', '8', '9']
Real: 3 - Predicted: ['1', '10', '2', '4', '6', '7', '8', '9']
Real: 3 - Predicted: ['1', '10', '4', '5', '6', '7', '8', '9']
Real: 3 - Predicted: ['1'

In [ ]:
X_test = np.array([np.array(i) for i in X_test])

classes = np.unique(Y).tolist()


def class_to_pos(current_class, real_class):
    if current_class == real_class:
        return 0
    return 1


for c, phi in model.PHI.items():
    regions = phi.predict(X_test, significance=SIGNIFICANCE)
    error = list(
        filter(
            lambda x: not x[1][x[0]],
            [(class_to_pos(Y_test[i], c), r) for i, r in enumerate(regions)],
        )
    )
    print(f"Error : {error}")

    error_p = np.array(
        [r[class_to_pos(Y_test[i], c)] for i, r in enumerate(regions)]
    ).mean()
    print(f"Error rate for class {c}: {error_p}")

    error_rate = len(error) / len(regions)
    res = np.array([r.sum() for r in regions]).mean()
    # print(f"results for class {c}: {res}")

Error : [(1, array([False, False])), (1, array([False, False])), (1, array([False, False])), (1, array([False, False])), (1, array([False, False])), (1, array([False, False])), (1, array([False, False]))]
Error rate for class 1: 0.825
Error : [(0, array([False, False])), (0, array([False, False])), (1, array([ True, False])), (1, array([False, False])), (1, array([False, False]))]
Error rate for class 10: 0.875
Error : [(1, array([False, False])), (1, array([False, False])), (1, array([ True, False])), (1, array([False, False]))]
Error rate for class 2: 0.9
Error : [(1, array([False, False])), (1, array([False, False])), (1, array([False, False])), (1, array([False, False])), (1, array([False, False])), (0, array([False, False])), (0, array([False, False])), (1, array([False, False])), (1, array([False, False]))]
Error rate for class 3: 0.775
Error : [(1, array([ True, False])), (1, array([False, False])), (1, array([False, False])), (0, array([False, False])), (0, array([False, False]